In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# This is for the plots not being cut off
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})



### Make a Prediction with Scikit-Learn
Steps:
1. Load data
1. Normalize
1. Train kNN classifier with training set
1. Test kNN classifier on test instance

In [8]:
def predict(df, predicting_attribute, test_data ):
    X_train = df.drop(predicting_attribute,axis=1)
    y_train = df[predicting_attribute]

    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_normalized = scaler.transform(X_train) # often combined into one step, using fit_transform()


    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train_normalized, y_train)

    X_test = pd.Series(test_data, index=df.columns.drop(predicting_attribute))
    X_test = scaler.transform([X_test])
    y_test_prediction = neigh.predict(X_test)

    return(y_test_prediction)

data = [[7, 7, "Bad"], [7, 4, "Bad"], [3, 4, "Good"], [1, 4, "Good"]]
df = pd.DataFrame(data, columns=["Acid durability (seconds)", "Strength (kg/square meter)", "Classification"])

result1 = predict(df, "Classification",pd.Series([1,7], index=df.columns.drop("Classification")))
result2 = predict(df, "Classification",pd.Series([9,7], index=df.columns.drop("Classification")))

print(result1)
print(result2)

['Good']
['Bad']
